In [3]:
import os
# Changing the working directory
os.chdir('../..')
os.getcwd()

'/home/javiermunoz/Universidad/MasterBDyDS/TFM/TFM_DSBD'

In [4]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("figure", figsize=(15, 5))

import os 
from src.config import DATA_PATH, SPLITS
import pathlib

/home/javiermunoz/anaconda3/envs/tf/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [5]:
gpu_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
subset_size = 10
subset = f"subset_{subset_size}_lsa_64"

In [7]:
subset_paths = {}
for sp in SPLITS:
    subset_paths[sp] = pathlib.Path(os.path.join(DATA_PATH, subset, sp))

## 2. Loading Video Dataset

https://www.tensorflow.org/tutorials/load_data/video

In [8]:
tf.keras.backend.clear_session()  

In [29]:
from src.frame_generator import FrameGenerator
import tensorflow as tf

In [10]:
fg = FrameGenerator(subset_paths['train'], 10, training=True)

frames, label = next(fg())
print(f"Shape: {frames.shape}")
print(f"Label: {label}")

Shape: (10, 224, 224, 3)
Label: 1


In [11]:
n_frames = 10
batch_size = 8

# We specify the shape of the output, such as, the Generator will produce a tuple of both
# videos, class, where the videos will have 3 channels, and the rest of dimensions will remain the same.
#   VideoShape -> (F, H, W, C)
output_signature = (tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(), dtype=tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['train'], n_frames, training=True),
                                          output_signature=output_signature)
# Also, we batchify the data, so the training process is not as memory consuming as if the whole dataset was 
# loaded into memory.
# VideoShape -> (B, F, H, W, C)
# train_ds = train_ds.batch(batch_size)
train_ds = train_ds.batch(batch_size)

# We reproduce this process for the validation and test splits too.
val_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['val'], n_frames),
                                        output_signature=output_signature)

val_ds = val_ds.batch(batch_size)
test_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['test'], n_frames),
                                         output_signature=output_signature)

test_ds = test_ds.batch(batch_size)

In [12]:
# Print the shapes of the data
train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames: {val_frames.shape}')
print(f'Shape of validation labels: {val_labels.shape}')

Shape of training set of frames: (8, 10, 224, 224, 3)
Shape of training labels: (8,)
Shape of validation set of frames: (8, 10, 224, 224, 3)
Shape of validation labels: (8,)


In [13]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)

In [14]:
(None, *train_frames.shape[2:])

(None, 224, 224, 3)

# 3. Model definition and Training

In [18]:
model = tf.keras.Sequential([
    # Construct the input layer with no definite frame size.
    tf.keras.layers.InputLayer(input_shape=(None, *train_frames.shape[2:])),
    # Reescale the values of the pixels (per frame)
    tf.keras.layers.Rescaling(scale=255),
    # We will construct 3 `ConvLSTM2D` layers with batch normalization,
    tf.keras.layers.ConvLSTM2D(
        filters=16,
        kernel_size=(3, 3),
        strides=(1,1),
        stateful=False,
        return_sequences=True,
        activation="tanh",
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(
        filters=8,
        kernel_size=(3, 3),
        strides=(1,1),
        stateful=False,
        return_sequences=False,
        activation="tanh",
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(
        filters=16,
        kernel_size=(3,3),                       
        strides=(1,1),
    ),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(units=10, activation='softmax')
])
model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy, 
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-03),
    metrics=['acc']
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, None, 224, 224, 3  0         
                             )                                   
                                                                 
 conv_lstm2d_4 (ConvLSTM2D)  (None, None, 222, 222, 1  11008     
                             6)                                  
                                                                 
 batch_normalization_4 (Batc  (None, None, 222, 222, 1  64       
 hNormalization)             6)                                  
                                                                 
 conv_lstm2d_5 (ConvLSTM2D)  (None, 220, 220, 8)       6944      
                                                                 
 batch_normalization_5 (Batc  (None, 220, 220, 8)      32        
 hNormalization)                                      

In [19]:
# Define some callbacks to improve training.
#early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

# Define modifiable training hyperparameters.
epochs = 80

# Fit the model to the training data using a generator.
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
    #callbacks=[early_stopping],
)

Epoch 1/80
50/50 [==============================] - 29s 531ms/step - loss: 3.2642 - acc: 0.1294 - val_loss: 2.3583 - val_acc: 0.0333
Epoch 2/80
50/50 [==============================] - 26s 524ms/step - loss: 2.2327 - acc: 0.1954 - val_loss: 2.2967 - val_acc: 0.1000
Epoch 3/80
50/50 [==============================] - 26s 523ms/step - loss: 2.0487 - acc: 0.2538 - val_loss: 2.2369 - val_acc: 0.1833
Epoch 4/80
50/50 [==============================] - 26s 524ms/step - loss: 1.8460 - acc: 0.3198 - val_loss: 2.0954 - val_acc: 0.2667
Epoch 5/80
50/50 [==============================] - 26s 526ms/step - loss: 1.7358 - acc: 0.3832 - val_loss: 1.9181 - val_acc: 0.2500
Epoch 6/80
50/50 [==============================] - 26s 528ms/step - loss: 1.6049 - acc: 0.4492 - val_loss: 1.7088 - val_acc: 0.4000
Epoch 7/80
50/50 [==============================] - 26s 532ms/step - loss: 1.5004 - acc: 0.4315 - val_loss: 1.5430 - val_acc: 0.4000
Epoch 8/80
50/50 [==============================] - 26s 528ms/step - 

In [20]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_ds, batch_size=batch_size)
print("test loss, test acc:", results)

Evaluate on test data
6/6 [==============================] - 12s 2s/step - loss: 0.6983 - acc: 0.7826
test loss, test acc: [0.6983227729797363, 0.782608687877655]


In [33]:
from src.plot_utils import plot_history

In [ ]:
plot_history(history)